# Лабораторная №1
## Задание

In [2]:
from scipy.stats import bernoulli, poisson, norm, expon

In [10]:
mean = 0
std_dev = 1
sample_size = 1000
normal_sample = norm.rvs(loc=mean, scale=std_dev, size=sample_size)

lambda_param = 3
poisson_sample = poisson.rvs(mu=lambda_param, size=sample_size)

p_param = 0.7
bernoulli_sample = bernoulli.rvs(p=p_param, size=sample_size)

exp_param = 2
exponential_sample = expon.rvs(scale=exp_param, size=sample_size)

In [11]:
exponential_sample.mean()

2.0946854547653047